# Google Colab 准备工作

In [126]:
'''
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
'''

'\n!apt-get install -y -qq software-properties-common python-software-properties module-init-tools\n!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null\n!apt-get update -qq 2>&1 > /dev/null\n!apt-get -y install -qq google-drive-ocamlfuse fuse\nfrom google.colab import auth\nauth.authenticate_user()\nfrom oauth2client.client import GoogleCredentials\ncreds = GoogleCredentials.get_application_default()\nimport getpass\n!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL\nvcode = getpass.getpass()\n!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}\n'

In [127]:
'''
!mkdir -p drive
!google-drive-ocamlfuse drive

from google.colab import drive
drive.mount('/content/drive')
'''

"\n!mkdir -p drive\n!google-drive-ocamlfuse drive\n\nfrom google.colab import drive\ndrive.mount('/content/drive')\n"

In [128]:
!pip install torchvision

In [129]:
!nvidia-smi

Tue Jun  8 10:23:38 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P0    41W / 250W |   2413MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [130]:
#!pip install mxnet-cu112 # 没用着呢
#!pip install minpy

In [131]:
import torch 
x = torch.empty(5,3)
print(x)

tensor([[7.1154e-25, 3.0771e-41, 2.0100e-01],
        [0.0000e+00,        nan, 0.0000e+00],
        [4.2963e+24, 2.0869e-10, 2.8175e+20],
        [4.1962e-08, 6.4832e-10, 6.8989e-07],
        [5.1622e-39, 3.0771e-41, 1.6270e-30]])


# main.py 前面import

In [132]:
# -*- coding: UTF-8 -*-
# Python
import os
import random

# Torch
import torch
import numpy as np  #注意这里改了，改成能用GPU的numpy了
import torch.nn as nn
import torch.optim as optim
from numpy.core._multiarray_umath import ndarray
from sklearn.cluster import KMeans
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
from torch.autograd import Variable
from torch.distributions import Categorical

# Torchvison
import torchvision.transforms as T
import torchvision.models as models
from torchvision.datasets import CIFAR100, CIFAR10
# from influence import *

# Utils
# import visdom
# from tqdm import tqdm

# Custom
#import models.resnet as resnet
#from config import *
#from data.sampler import SubsetSequentialSampler

# import copy

#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"


# config.py

In [133]:
##
# Learning Loss for Active Learning
NUM_TRAIN = 50000 # N
NUM_VAL   = 50000 - NUM_TRAIN
BATCH     = 128  # 128 # B  注意，这里本来是是128
SUBSET    = 25000 # M  # 本来是25000
ADDENDUM  = 2500  # K   # 本来是2500

MARGIN = 1.0  # xi
WEIGHT = 1.0  # 1.0 # lambda

TRIALS = 1
CYCLES = 7   # 本来应该是7

EPOCH = 200 # 本来是200
LR = 0.1    # 0.1 for SGD
MILESTONES = [160]
EPOCHL = 120 # After 120 epochs, stop the gradient from the loss prediction module propagated to the target model

MOMENTUM = 0.9
WDECAY = 5e-4

CLUSTER_NUMS = 10 # 新加的，聚类的cluster 数量
CLUSTER_MAX_ITER = 10000 # 新加的，聚类的迭代次数，本来是10000


''' CIFAR-10 | ResNet-18 | 93.6%
NUM_TRAIN = 50000 # N
NUM_VAL   = 50000 - NUM_TRAIN
BATCH     = 128 # B
SUBSET    = NUM_TRAIN # M
ADDENDUM  = NUM_TRAIN # K

MARGIN = 1.0 # xi
WEIGHT = 0.0 # lambda

TRIALS = 1
CYCLES = 1

EPOCH = 50
LR = 0.1
MILESTONES = [25, 35]
EPOCHL = 40

MOMENTUM = 0.9
WDECAY = 5e-4
'''


' CIFAR-10 | ResNet-18 | 93.6%\nNUM_TRAIN = 50000 # N\nNUM_VAL   = 50000 - NUM_TRAIN\nBATCH     = 128 # B\nSUBSET    = NUM_TRAIN # M\nADDENDUM  = NUM_TRAIN # K\n\nMARGIN = 1.0 # xi\nWEIGHT = 0.0 # lambda\n\nTRIALS = 1\nCYCLES = 1\n\nEPOCH = 50\nLR = 0.1\nMILESTONES = [25, 35]\nEPOCHL = 40\n\nMOMENTUM = 0.9\nWDECAY = 5e-4\n'

# device设置成CPU或GPU 

In [134]:
#
# 加一个device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())

True


# sampler.py

In [135]:
class SubsetSequentialSampler(torch.utils.data.Sampler):
    r"""Samples elements sequentially from a given list of indices, without replacement.

    Arguments:
        indices (sequence): a sequence of indices
    """

    def __init__(self, indices):
        self.indices = indices

    def __iter__(self):
        return (self.indices[i] for i in range(len(self.indices)))

    def __len__(self):
        return len(self.indices)



# main.py 里面的data

In [136]:
##
# Data
train_transform = T.Compose([
    T.RandomHorizontalFlip(),
    T.RandomCrop(size=32, padding=4),
    T.ToTensor(),
    T.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
])

test_transform = T.Compose([
    T.ToTensor(),
    T.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
])

cifar10_train = CIFAR10('../cifar10', train=True, download=True, transform=train_transform)  # specify data path here
cifar10_unlabeled = CIFAR10('../cifar10', train=True, download=True, transform=test_transform)
cifar10_test = CIFAR10('../cifar10', train=False, download=True, transform=test_transform)

##
# Train Utils
iters = 0


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


# resnet.py

In [137]:
#import torch
#import torch.nn as nn
#import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)
        self.linear1 = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out1 = self.layer1(out)
        out2 = self.layer2(out1)
        out3 = self.layer3(out2)
        out4 = self.layer4(out3)
        out5 = F.avg_pool2d(out4, 4)
        out5 = out5.view(out5.size(0), -1)     # [128, 512]
        out = self.linear(out5)
        out_cons = self.linear1(out5)

        return out, out_cons, out5, [out1, out2, out3, out4]


def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])

def ResNet18_student():
    return ResNet(BasicBlock, [1,1,1,1])

def ResNet34():
    return ResNet(BasicBlock, [3,4,6,3])

def ResNet50():
    return ResNet(Bottleneck, [3,4,6,3])

def ResNet101():
    return ResNet(Bottleneck, [3,4,23,3])

def ResNet152():
    return ResNet(Bottleneck, [3,8,36,3])



# main.py 里面的train_epoch函数

In [138]:
#
def train_epoch(models, criterion, optimizers, dataloaders, epoch, epoch_loss):
    models['backbone'].train()
    global iters

    for data in dataloaders['train']:
        inputs = data[0].to(device)
        labels = data[1].to(device)
        iters += 1

        optimizers['backbone'].zero_grad()

        scores, _, _, features_list = models['backbone'](inputs)
        target_loss = criterion(scores, labels)

        m_backbone_loss = torch.sum(target_loss) / target_loss.size(0)
        loss = m_backbone_loss
        loss.backward()
        optimizers['backbone'].step()


# main.py 里面的test函数

In [139]:
#
def test(models, dataloaders, mode='val'):
    assert mode == 'val' or mode == 'test'
    models['backbone'].eval()

    total = 0
    correct = 0
    with torch.no_grad():
        for (inputs, labels) in dataloaders[mode]:
            #inputs = inputs.cuda()这才是prcharm原来的
            inputs = inputs.to(device)
            #labels = labels.cuda()这才是prcharm原来的
            labels = labels.to(device)
            scores, _, _, _ = models['backbone'](inputs)
            _, preds = torch.max(scores.data, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()

    return 100 * correct / total

# main.py里面的train函数

In [140]:
#
def train(models, criterion, optimizers, schedulers, dataloaders, num_epochs, epoch_loss, cycle):
    print('>> Train a Model...')
    best_acc = 0.
    checkpoint_dir = os.path.join('./cifar10', 'train', 'weights')
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

    for epoch in range(num_epochs):

        train_epoch(models, criterion, optimizers, dataloaders, epoch, epoch_loss)
        schedulers['backbone'].step()

        # Save a checkpoint
        if epoch % 20 == 0 or epoch == EPOCH - 1:
            acc = test(models, dataloaders, 'test')
            if best_acc < acc:
                best_acc = acc
                torch.save({
                    'epoch': epoch + 1,
                    'state_dict_backbone': models['backbone'].state_dict()
                    # 'state_dict_module': models['module'].state_dict()
                },
                    '%s/active_resnet18_cifar10.pth' % (checkpoint_dir))
            print('Cycle:', cycle + 1, 'Epoch:', epoch + 1, "---", 'Val Acc: {:.3f} \t Best Acc: {:.3f}'.format(acc, best_acc),flush=True)
            
    print('>> Finished.')


# Normalize distance

In [141]:
#
# Normalized distance
def normalize_distance(input_x, cores, result, min_distance):
    average_distance = np.empty(CLUSTER_NUMS, dtype=np.float32) #每个cluster里所有样本距离core的平均距离
    standard_dev = np.empty(CLUSTER_NUMS, dtype=np.float32) #每个cluster里所有样本距离core的 标准差
    line_num, column_num = input_x.shape
    normalized_distance = np.empty((line_num, 1), dtype=np.float32)
    for i in range(CLUSTER_NUMS):  # 遍历质心集
        if len(input_x[result == i]) != 0:
            items = input_x[result == i]       # 找出对应当前质心的子样本集
            cores[i] = np.mean(items, axis=0)
                    
            d_abs = np.abs(items - cores[i])
            d_square = np.square(d_abs)
            total_distance = np.sqrt(np.sum(d_square, axis=1)) # 在这个cluster中每个样本距离自己core的距离
            average_distance[i] = np.mean(total_distance, axis = 0) # 这个cluster中样本距离core的平均距离,一个数
            standard_dev[i] = np.std(total_distance, axis=0) #标准差, 一个数
            
            normalized_distance[result == i] = (min_distance[result == i] - average_distance[i])/standard_dev[i]
            normalized_distance_torch = torch.from_numpy(normalized_distance)
            #print(normalized_distance_torch)
    
    print("standard_dev", standard_dev)
    
    return normalized_distance_torch, average_distance, standard_dev


# main.py里面的get_uncertainty函数

In [142]:
#
def get_uncertainty(models, unlabeled_loader):
    models['backbone'].eval()
    #uncertainty = torch.tensor([]).to(device)  #这是原来版本，聚类先不用GPU，用CPU可以用numpy
    uncertainty = torch.tensor([])
    all_features = torch.tensor([]).to(device)

    with torch.no_grad():
        print(">> Start clustering:")
        for (inputs, labels) in unlabeled_loader:
            #inputs = inputs.cuda()这才是pycharm原来的
            inputs = inputs.to(device)
            
            scores, _, total_feature, features = models['backbone'](inputs)
            # total_feature: [SUBSET,512], SUBSET是行 = 128，是batch_size，也就是图片个数，512是列，是resnet展平的feature个数
            
            # 这是原来版本，计算关键点 因为KMeans.fix(X[,y) X是需要2D 而不是1D
            all_features = torch.cat((all_features, total_feature), axis = 0)

        input_x = all_features.cpu().numpy()
        line_num, column_num = input_x.shape  # line_num：样本数量，column_num：每个样本的feature里个数

        result = np.empty(line_num, dtype=np.int)  # line_num个样本的最近的cluster index
        distance = np.empty((line_num, CLUSTER_NUMS), dtype=np.float32)  

        # 从line_num个数据样本中不重复地随机选择k个样本作为质心,cores
        cores = input_x[np.random.choice(np.arange(line_num), CLUSTER_NUMS, replace=False)]
        min_distance = np.empty((line_num, 1), dtype=np.float32) #距离最近的cluster的质心的距离
        second_min_distance = np.empty((line_num, 1), dtype=np.float32) #距离第二近的cluster的质心的距离

        iter = 0 #迭代次数
        while True:  # 迭代聚类计算,max iteration是10000次，质心都不变时停止
            iter = iter + 1
            d = np.square(np.repeat(input_x, CLUSTER_NUMS, axis=0).reshape(line_num, CLUSTER_NUMS, column_num) - cores)
            distance = np.sqrt(np.sum(d, axis=2))  # ndarray(line_num, k)，每个样本距离 k（CLUSTER_NUM）个质心的距离，共有line_num行

            index_min = np.argmin(distance, axis=1)  # 每个样本距离最近的质心索引序号 [line_num, 1]
            if (index_min == result).all() or iter == CLUSTER_MAX_ITER:  #质心不变时停止
                break;

            result[:] = index_min  # 重新分类 [line_num, 1] 这些图片分别属于哪个cluster, index
            for i in range(CLUSTER_NUMS):  # 遍历质心集
                if len(input_x[result == i]) != 0:
                    items = input_x[result == i]       # 找出对应当前质心的子样本集
                    cores[i] = np.mean(items, axis=0)  # 以子样本集的均值作为当前质心的位置
        
        index_sorted = np.argsort(distance)
        index_second_min  = index_sorted[:,1] #line_num个样本的第二近的cluster indexs

        sorted_distance = np.sort(distance) # 将矩阵的每一行升序排列
        min_distance = np.abs(sorted_distance[:, 0].reshape(line_num, 1)) #每个样本距离自己属于的cluster质心的距离
        second_min_distance = np.abs(sorted_distance[:, 1].reshape(line_num, 1)) 

        average_distance = np.empty(CLUSTER_NUMS, dtype=np.float32) #每个cluster里所有样本距离core的平均距离
        standard_dev = np.empty(CLUSTER_NUMS, dtype=np.float32) #每个cluster里所有样本距离core的 标准差
        normalized_second_min_dist = np.empty((line_num, 1), dtype=np.float32)
        #
        # Normalized distance
        normalized_min_dist_torch, average_distance, standard_dev = normalize_distance(input_x, cores, result, min_distance)
        
        for i in range(CLUSTER_NUMS):  # 遍历质心集
            normalized_second_min_dist[index_second_min == i] = (second_min_distance[index_second_min == i] - average_distance[i])/standard_dev[i]
                
        normalized_second_min_dist_torch = torch.from_numpy(normalized_second_min_dist)
        normalized_dist_differ_torch = normalized_second_min_dist_torch - normalized_min_dist_torch
        
        uncertainty = 10.00/normalized_dist_differ_torch

    print(">> Clustering Over:")
    return uncertainty.cpu()


# main.py 里面的main函数

In [143]:

##
# Main
if __name__ == '__main__':

    for trial in range(TRIALS):
        # Initialize a labeled dataset by randomly sampling K=ADDENDUM=1,000 data points from the entire dataset.
        indices = list(range(NUM_TRAIN))
        random.shuffle(indices)
        START = 2 * ADDENDUM
        labeled_set = indices[:START]
        unlabeled_set = indices[START:]

        train_loader = DataLoader(cifar10_train, batch_size=BATCH,  # BATCH
                                  sampler=SubsetRandomSampler(labeled_set),
                                  pin_memory=True)
        test_loader = DataLoader(cifar10_test, batch_size=BATCH)

        dataloaders = {'train': train_loader, 'test': test_loader}

        # Model
        #backbone_net = resnet.ResNet18().cuda() #记住这才是原本pycharm里面的
        backbone_net = ResNet18().to(device)
        
        models = {'backbone': backbone_net}
        torch.backends.cudnn.benchmark = True

        # Active learning cycles
        for cycle in range(CYCLES):
            # Loss, criterion and scheduler (re)initializa tion
            criterion = nn.CrossEntropyLoss(reduction='none')

            optim_backbone = optim.SGD(models['backbone'].parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WDECAY)

            sched_backbone = lr_scheduler.MultiStepLR(optim_backbone, milestones=MILESTONES)

            optimizers = {'backbone': optim_backbone}
            schedulers = {'backbone': sched_backbone}

            # Training and test
            train(models, criterion, optimizers, schedulers, dataloaders, EPOCH, EPOCHL, cycle)
            acc = test(models, dataloaders, mode='test')
            print('Trial {}/{} || Cycle {}/{} || Label set size {}: Test acc {}'.format(trial + 1, TRIALS, cycle + 1,
                                                                                        CYCLES, len(labeled_set), acc),
                                                                                         flush=True)

            ##
            #  Update the labeled dataset via loss prediction-based uncertainty measurement

            # Randomly sample 10000 unlabeled data points
            random.shuffle(unlabeled_set)
            subset = unlabeled_set[:SUBSET]

            # Create unlabeled dataloader for the unlabeled subset
            unlabeled_loader = DataLoader(cifar10_unlabeled, batch_size=BATCH,
                                          sampler=SubsetSequentialSampler(subset),
                                          # more convenient if we maintain the order of subset
                                          pin_memory=True)

            # Measure uncertainty of each data points in the subset
            uncertainty = get_uncertainty(models, unlabeled_loader)
            #print("main uncertainty: ", uncertainty)
            uncertainty = uncertainty.T
            # Index in ascending order
            arg = np.argsort(uncertainty).numpy().tolist()
            #print("main arg: ", arg) 
            
            # Update the labeled dataset and the unlabeled dataset, respectively
            labeled_set += list(torch.tensor(subset)[arg][-ADDENDUM:].numpy())  # select largest loss
            unlabeled_set = list(torch.tensor(subset)[arg][:-ADDENDUM].numpy()) + unlabeled_set[SUBSET:]

            # labeled_set += list(torch.tensor(subset)[arg][:ADDENDUM].numpy())  # select smallest influence
            # unlabeled_set = list(torch.tensor(subset)[arg][ADDENDUM:].numpy()) + unlabeled_set[SUBSET:]

            # Create a new dataloader for the updated labeled dataset
            dataloaders['train'] = DataLoader(cifar10_train, batch_size=BATCH,  # BATCH
                                              sampler=SubsetRandomSampler(labeled_set),
                                              pin_memory=True)

        # Save a checkpoint
        torch.save({
            'trial': trial + 1,
            'state_dict_backbone': models['backbone'].state_dict()
            # 'state_dict_module': models['module'].state_dict()
        },
            './cifar10/train/weights/active_resnet18_cifar10_trial{}.pth'.format(trial))

        print('---------------------------Current Trial is done-----------------------------',flush=True)


>> Train a Model...
Cycle: 1 Epoch: 1 --- Val Acc: 17.370 	 Best Acc: 17.370
Cycle: 1 Epoch: 21 --- Val Acc: 46.780 	 Best Acc: 46.780
Cycle: 1 Epoch: 41 --- Val Acc: 60.910 	 Best Acc: 60.910
Cycle: 1 Epoch: 61 --- Val Acc: 68.090 	 Best Acc: 68.090
Cycle: 1 Epoch: 81 --- Val Acc: 65.240 	 Best Acc: 68.090
Cycle: 1 Epoch: 101 --- Val Acc: 70.770 	 Best Acc: 70.770
Cycle: 1 Epoch: 121 --- Val Acc: 70.640 	 Best Acc: 70.770
Cycle: 1 Epoch: 141 --- Val Acc: 68.550 	 Best Acc: 70.770
Cycle: 1 Epoch: 161 --- Val Acc: 78.200 	 Best Acc: 78.200
Cycle: 1 Epoch: 181 --- Val Acc: 78.520 	 Best Acc: 78.520
Cycle: 1 Epoch: 200 --- Val Acc: 78.910 	 Best Acc: 78.910
>> Finished.
Trial 1/1 || Cycle 1/7 || Label set size 5000: Test acc 78.91
>> Start clustering:
standard_dev [0.7856425  0.67395145 0.72444046 0.7393518  0.68609494 0.68691105
 0.71155804 0.7247792  0.7413927  0.75690526]
>> Clustering Over:
>> Train a Model...
Cycle: 2 Epoch: 1 --- Val Acc: 75.940 	 Best Acc: 75.940
Cycle: 2 Epoch: 21